In [25]:
import numpy as np
import csv
import math
import xlrd
import pandas as pd
from tqdm import tqdm


In [2]:
def ReadMyCsv(SaveList, fileName):
    csv_reader = csv.reader(open(fileName))
    for row in csv_reader:
        SaveList.append(row)
    return

def StorFile(data, fileName):
    with open(fileName, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    return

In [3]:
# load files
FinalAllDisease = []
ReadMyCsv(FinalAllDisease, "side_effect.csv")#将adr添加到FinalAllDisease中

len(FinalAllDisease)#994个adr
FinalAllDisease[0]

# ReadMyCsv(FinalAllDisease, "disease.csv")
FinalAllDisease = np.array(FinalAllDisease)[:, 0]
print(len(FinalAllDisease))
print(FinalAllDisease[1])
# FinalAllDisease = FinalAllDisease[0:100]


DiseaseMeSHTreeStructure = []
# ReadMyCsv(DiseaseMeSHTreeStructure, "MeSHTreeStructureLow.csv")
# print(len(DiseaseMeSHTreeStructure))
# print(DiseaseMeSHTreeStructure[0])

data1 = xlrd.open_workbook('ADR_Drug.xlsx')

table1 = data1.sheet_by_name('Sheet 1')
table1.nrows
table1.ncols
#table1的变量名
#第三类是ADReCS的id，第四列是小写的adr

994
abdominal distension


4

In [4]:
for i in range(table1.nrows):
    x = table1.cell(i, 3).value
    y = table1.cell(i, 2).value
    y = y.split('.')
    if len(y) == 4:
        y1 = 'a'+y[0]
        y2 = '0'+y[1]
        y3 = '0'+y[2]
        y = y1+'.' + y2 + '.' + y3 + '.' + y[3]
    if len(y) == 3:
        y1 = 'a' + y[0]
        y2 = '0' + y[1]
        y3 = '0' + y[2]
        y = y1 + '.' + y2 + '.' + y3
    if len(y) == 2:
        y1 = 'a' + y[0]
        y2 = '0' + y[1]
        y = y1 + '.' + y2
    if len(y) == 1:
        y = 'a' + y[0]
    DiseaseMeSHTreeStructure.append([x, y])
DiseaseMeSHTreeStructure[0]

AllDisease = FinalAllDisease
mesh = DiseaseMeSHTreeStructure

In [5]:
DiseaseAndMeshID = []
counter1 = 0
while counter1 < len(AllDisease):
    DiseaseAndMeshPair = []
    DiseaseAndMeshID.append(DiseaseAndMeshPair)
    DiseaseAndMeshID[counter1].append(AllDisease[counter1])
    counter2 = 0
    flag = 0
    while counter2 < len(mesh):
        if (mesh[counter2][0] == DiseaseAndMeshID[counter1][0]) & (flag == 1):
            DiseaseAndMeshID[counter1][1].append(mesh[counter2][1]) #将疾病对应的mesh id添加到DiseaseAndMeshID中
        if (mesh[counter2][0] == DiseaseAndMeshID[counter1][0]) & (flag == 0):
            MeshID = []
            MeshID.append(mesh[counter2][1])
            DiseaseAndMeshID[counter1].append(MeshID)
            flag = 1
        if (counter2 == len(mesh) - 1) & (len(DiseaseAndMeshID[counter1]) == 1):
            DiseaseAndMeshID[counter1].append(0)
        counter2 = counter2 + 1
    counter1 = counter1 + 1
print('DiseaseAndMeshID')
print(len(DiseaseAndMeshID))


DiseaseAndMeshID
994


In [6]:
DAGs = []
counter1 = 0
while counter1 < len(AllDisease):
    group = []
    group.extend(DiseaseAndMeshID[counter1])
    group.append(0)
    group1 = []
    group1.append(group)
    DAGs.append(group1)
    counter1 = counter1 + 1
print('len(DAGs)', len(DAGs))

DAGs[0]

len(DAGs) 994


[['abdominal discomfort', ['a07.001.006.001'], 0]]

In [7]:
for counter in tqdm(range(len(DAGs))):
    #print(DAGs[counter][0][1])
    #print(len(DAGs[counter]))

    if DAGs[counter][0][1] == 0:
        counter = counter + 1
        continue
    counter1 = 0
    while counter1 < len(DAGs[counter]):  #################
        counter2 = 0
        while counter2 < len(DAGs[counter][counter1][1]):
            layer = DAGs[counter][counter1][2]  #######################
            # if len(DAGs[0][counter1][1][counter2]) <= 3:
            #     break
            #print(len(DAGs[counter][counter1][1]))
            #print(layer)
            #print(len(DAGs[counter][counter1][1][counter2]))
            if len(DAGs[counter][counter1][1][counter2]) > 3:  ####################
                NID = DAGs[counter][counter1][1][counter2]  #####################
                #print(NID)
                L = len(NID)
                NID = NID[0:L - 4]
                #print(NID)
                counter3 = 0
                flag = 1
                while counter3 < len(mesh):
                    #print(len(mesh))
                    #print(mesh[counter3][1])
                    #print(mesh[counter3][0])
                    if NID == mesh[counter3][1]:
                        flag = 0
                        num = counter3
                        DiseaseName = mesh[counter3][0]
                        break
                    counter3 = counter3 + 1

                flag2 = 0
                counter5 = 0
                while counter5 < len(DAGs[counter]):
                    if DAGs[counter][counter5][0] == DiseaseName:
                        flag2 = 1
                        break
                    counter5 = counter5 + 1

                if flag == 0:
                    if flag2 == 0:
                        counter6 = 0
                        IDGroup = []
                        while counter6 < len(mesh):
                            if DiseaseName == mesh[counter6][0]:
                                IDGroup.append(mesh[counter6][1])
                            counter6 = counter6 + 1
                        DiseasePoint = []
                        layer = layer + 1
                        DiseasePoint.append(DiseaseName)
                        DiseasePoint.append(IDGroup)
                        DiseasePoint.append(layer)
                        DAGs[counter].append(DiseasePoint)

            counter2 = counter2 + 1
        counter1 = counter1 + 1

100%|██████████| 994/994 [00:10<00:00, 93.19it/s] 


In [8]:
DiseaseValue = []
counter = 0
while counter < len(AllDisease):
    if DAGs[counter][0][1] == 0:
        DiseaseValuePair = []
        DiseaseValuePair.append(AllDisease[counter])
        DiseaseValuePair.append(0)
        DiseaseValue.append(DiseaseValuePair)
        counter = counter + 1
        continue
    counter1 = 0
    DV = 0
    while counter1 < len(DAGs[counter]):
        DV = DV + math.pow(0.5, DAGs[counter][counter1][2])
        counter1 = counter1 + 1
    DiseaseValuePair = []
    DiseaseValuePair.append(AllDisease[counter])
    DiseaseValuePair.append(DV)
    DiseaseValue.append(DiseaseValuePair)
    counter = counter + 1
print('len(DiseaseValue)', len(DiseaseValue))
DiseaseValue[0]

len(DiseaseValue) 994


['abdominal discomfort', 1.875]

In [9]:
SameValue1 = []
for counter in tqdm(range(len(AllDisease))):
    RowValue = []
    if DiseaseValue[counter][1] == 0:
        counter1 = 0
        while counter1 < len(AllDisease):
            RowValue.append(0)
            counter1 = counter1 + 1
        SameValue1.append(RowValue)
        counter = counter + 1
        continue
    counter1 = 0
    while counter1 < len(AllDisease):
        if DiseaseValue[counter1][1] == 0:
            RowValue.append(0)
            counter1 = counter1 + 1
            continue
        DiseaseAndDiseaseSimilarityValue = 0
        counter2 = 0
        while counter2 < len(DAGs[counter]):
            counter3 = 0
            while counter3 < len(DAGs[counter1]):
                if DAGs[counter][counter2][0] == DAGs[counter1][counter3][0]:
                    DiseaseAndDiseaseSimilarityValue = DiseaseAndDiseaseSimilarityValue + math.pow(0.5, DAGs[counter][counter2][2]) + math.pow(0.5, DAGs[counter1][counter3][2]) #自己和自己的全部节点相同，对角线即DiseaseValue的两倍
                counter3 = counter3 + 1
            counter2 = counter2 + 1
        RowValue.append(DiseaseAndDiseaseSimilarityValue)
        counter1 = counter1 + 1
    SameValue1.append(RowValue)
print('SameValue1')
len(SameValue1)

100%|██████████| 994/994 [00:08<00:00, 122.39it/s]

SameValue1


994

In [10]:
DiseaseSimilarityModel1 = []
for counter in tqdm(range(len(AllDisease))):
    RowValue = []
    if DiseaseValue[counter][1] == 0:
        counter1 = 0
        while counter1 < len(AllDisease):
            RowValue.append(0)
            counter1 = counter1 + 1
        DiseaseSimilarityModel1.append(RowValue)
        counter = counter + 1
        continue
    counter1 = 0
    while counter1 < len(AllDisease):
        if DiseaseValue[counter1][1] == 0:
            RowValue.append(0)
            counter1 = counter1 + 1
            continue
        value = SameValue1[counter][counter1] / (DiseaseValue[counter][1] + DiseaseValue[counter1][1])
        RowValue.append(value)
        counter1 = counter1 + 1
    DiseaseSimilarityModel1.append(RowValue)

print('DiseaseSimilarityModel1，', len(DiseaseSimilarityModel1))
print('DiseaseSimilarityModel1[0]', len(DiseaseSimilarityModel1[0]))
DiseaseSimilarityModel1[0][0:10]

100%|██████████| 994/994 [00:00<00:00, 2326.63it/s]

DiseaseSimilarityModel1， 994
DiseaseSimilarityModel1[0] 994


[1.0, 0.2, 0.2, 0.2, 0.2, 0.0, 0.0, 0.2, 0.0, 0.0]

In [11]:
counter = 0
while counter < len(DiseaseSimilarityModel1):
    Row = []
    Row.append(AllDisease[counter])
    Row.extend(DiseaseSimilarityModel1[counter])
    DiseaseSimilarityModel1[counter] = Row
    counter = counter + 1

In [22]:
adr_similar=[[i[0],i[1:]] for i in DiseaseSimilarityModel1]

In [26]:
adr_similar_df=pd.DataFrame(adr_similar,columns=['adr_name','adr_similar'])

In [28]:
adr_similar_df.head

<bound method NDFrame.head of                  adr_name                                        adr_similar
0    abdominal discomfort  [1.0, 0.2, 0.2, 0.2, 0.2, 0.0, 0.0, 0.2, 0.0, ...
1    abdominal distension  [0.2, 1.0, 0.2, 0.2, 0.2, 0.0, 0.0, 0.2, 0.0, ...
2          abdominal pain  [0.2, 0.2, 1.0, 0.4666666666666667, 0.46666666...
3    abdominal pain lower  [0.2, 0.2, 0.4666666666666667, 1.0, 0.46666666...
4    abdominal pain upper  [0.2, 0.2, 0.4666666666666667, 0.4666666666666...
..                    ...                                                ...
989   withdrawal syndrome  [0.0, 0.0, 0.0, 0.0, 0.0, 0.05405405405405406,...
990       wound infection  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.378...
991       wound secretion  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
992         xerophthalmia  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
993               yawning  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

[994 rows x 2 columns]>

In [30]:
adr_similar_df.to_csv('adr_similar_df.csv')

利用本文件产生了adr的语义相似特征，储存为csv然后通过G，Z等人的研究里面的其他数据集进行拼接